# Hacettepe University BBM-409 Course Assignment 1 

Student Name: Mehmet Emin TUNCER
Student Number: 21591022




# Report



PART#1.Theory Questions
    #k-Nearest Neighbor Classiﬁcation:
    
    1) As the 'K' value in the K-NN algorithm increases, the algorithm usually does not learn anything new. It usually tries to reach a conclusion by classifying the data. In addition, increasing the 'K' value will cause the algorithm to run slowly when calculating the distance.
    
    
    4)F-T-T
    



PART#2:
#1.Problem:
    In this assignment, we were asked to separate the file, which was used and the users' votes for the movies, as test and train data, and to estimate how much a user voted for a movie.
    We expected the best results using KNN and weighted KNN algorithms   to find the algorithm that works best with the help of the MAE value..

#2. Solution:
    




In [28]:
import numpy as np
import pandas as pd
import os
from numpy import dot
from numpy.linalg import norm
import operator
class User:
    userID=0
    userIndex=0
    movies={}
    watchedMovies={}
    kNN=[]
    votedMovies=[]
    mAE=0

class Neighbour:
    neighbourID=0;
    distance=0;
class Movies:
    movieID=0
    movieIndex=0


def findAllUsers(userList, train_pd,userIDs):
    j=0
    for i in range(train_pd.shape[0]):
        if train_pd.iloc[i, 0] not in userIDs:
            user=User()
            user.userID=train_pd.iloc[i, 0]
            user.userIndex=j
            userIDs.append(train_pd.iloc[i, 0])
            userList.append(user)
            j=j+1




        
        
def findWatchedMovies(userList,dataFrame):
    for i in range(dataFrame.shape[0]):
        user=User()
        user.userID=i
        userList.append(user)
        for j in range(dataFrame.shape[1]):
            if dataFrame.iloc[i,j]!=0:
                user.watchedMovies[j]=dataFrame.iloc[i,j]



###To determine how to split the train file into test (25%) and train (75%).
def splitToTrainAndTesT(dataFrame,size):
    size=int((dataFrame.shape[0]/100)*25)

    
    
##This function finds all neighbors of users.  
def findKNNs(testData_reset,trainData_reset,nearest_NeighborDic,cosineSimData,testUsers,weighted_kNN_Dic):
    for i in range(cosineSimData.shape[0]):

        user=User()
        user.userID=testData_reset.iloc[i, 0]
        nearest_NeighborDic[testData_reset.iloc[i, 0]]={}
        weighted_kNN_Dic[testData_reset.iloc[i, 0]]={}
        user.kNN=[]
        user.votedMovies=[]
        for j in range(cosineSimData.shape[1]):
            neighbour=Neighbour()
            neighbour.neighbourID=trainData_reset.iloc[j,0]
            neighbour.distance=cosineSimData[i,j]
            user.kNN.append(neighbour)
            weighted_kNN_Dic[testData_reset.iloc[i,0]][trainData_reset.iloc[j,0]]=cosineSimData[i,j]
            nearest_NeighborDic[testData_reset.iloc[i,0]][cosineSimData[i,j]]=trainData_reset.iloc[j,0]

        testUsers.append(user)

def sortKNNS(testUsers):
    for i in range(len(testUsers)):
        testUsers[i].kNN = sorted(testUsers[i].kNN, key=operator.attrgetter('distance'),reverse=True)


#To change the 'K' value in K-NN to see the effect on the algorithm.        
def decideKNNs(tesUsers,x):
    for i in range(len(testUsers)):
        newKNN=testUsers[i].kNN[:x]
        testUsers[i].kNN=newKNN


##To estimate how much the given user has voted for the desired movie by using close neighbors.
def guessRate(user,movieID,userVotedMovies):
    rating=0
    count=0
    for neigh in user.kNN:
        if movieID in userVotedMovies[neigh.neighbourID]:
            rating=rating+userVotedMovies[neigh.neighbourID][movieID]
            count=count+1
        else:
            count=count+1
    return rating/count

##To estimate how much the given user has voted for the desired movie by using weighted k-NN

def guessRate_with_KNN(user,movieID,weighted_kNN_Dic):
    rating = 0
    count = 0
    distance=1
    for neigh in user.kNN:
        if movieID in userVotedMovies[neigh.neighbourID]:
            distance1=weighted_kNN_Dic[user.userID][neigh.neighbourID]

            distance=distance+distance1
            rating = rating + userVotedMovies[neigh.neighbourID][movieID]*distance1
            count=count+1
        else:
            count=count+1

    return rating/distance

#To find MAE with K-NN
def findMAE(testUsers,userVotedMovies):
    meanAE=0
    userRate=0
    guessedRate=0
    rateDiff=0
    countVotes=0
    rated=0
    for i in range(len(testUsers)):

        for j in range(len(testUsers[i].votedMovies)):
            countVotes=countVotes+1

            userRate=userVotedMovies[testUsers[i].userID][testUsers[i].votedMovies[j]]
            guessedRate=guessRate(testUsers[i],testUsers[i].votedMovies[j],userVotedMovies)

            rateDiff = userRate - guessedRate

            rateDiff = abs(rateDiff)
            rated = rated + rateDiff


    meanAE=rated/countVotes
    return meanAE


def findMAE_with_Weighted_KNN(testUsers,userVotedMovies,weighted_kNN_Dic):
    meanAE = 0
    userRate = 0
    guessedRate = 0
    rateDiff = 0
    countVotes = 0
    rated=0
    for i in range(len(testUsers)):
        for j in range(len(testUsers[i].votedMovies)):

            countVotes=countVotes+1
            userRate=userVotedMovies[testUsers[i].userID][testUsers[i].votedMovies[j]]
            guessedRate=guessRate_with_KNN(testUsers[i],testUsers[i].votedMovies[j],weighted_kNN_Dic)

            rateDiff = userRate - guessedRate
            rateDiff = abs(rateDiff)
            rated = rated + rateDiff

    meanAE=rated/countVotes
    return meanAE


def findUserVoted(testUsers,userVotedMovies):
    for i in range(len(testUsers)):
        list=[]
        for k in userVotedMovies[testUsers[i].userID]:

            list.append(k)
            testUsers[i].votedMovies.append(k)






weighted_kNN_Dic={}
trainUsers=[]
testUsers=[]

movieList=[]
movieIDs=[]


#####I read the rating table, delete unnecessary rows, and eventually create a pivot table of userId, movieId, and ratings.

combined_DataFrame = pd.read_csv("ratings_train.csv")
movies_pd=pd.read_csv("movies.csv")

del movies_pd["genres"]
del movies_pd["title"]
del combined_DataFrame["timestamp"]

combined_DataFrame=combined_DataFrame.pivot(index="userId",columns="movieId")
combined_DataFrame=combined_DataFrame.fillna(0)




split=int((combined_DataFrame.shape[0]/100)*25)

combined_DataFrame=combined_DataFrame.iloc[np.random.permutation(len(combined_DataFrame))]
combined_DataFrame_indexReset=combined_DataFrame.reset_index()


###I separated the test and train data.

testData=combined_DataFrame.iloc[:split,:]
testData_indexReset=testData.reset_index()


trainData=combined_DataFrame.iloc[split:,:]
trainData_indexReset=trainData.reset_index()


train_data_pd =pd.read_csv("ratings_train.csv",usecols = ["userId","movieId","rating"])
userIDsList=train_data_pd['userId'].tolist()
moiveIDsList=train_data_pd['movieId'].tolist()
ratingsList=train_data_pd['rating'].tolist()



userVotedMovies={}
for i in range(len(userIDsList)):
    userVotedMovies[userIDsList[i]]={}
for i in range(len(userIDsList)):
    userVotedMovies[userIDsList[i]][moiveIDsList[i]]=ratingsList[i]



cosineSimData = dot(testData, trainData.transpose())/(norm(testData)*norm(trainData.transpose()))

nearest_NeighborDic={}
final_nearest_NeighborDic={}

for i in range(testData_indexReset.shape[0]):
    nearest_NeighborDic[testData_indexReset.iloc[i,0]]={}


findKNNs(testData_indexReset,trainData_indexReset,nearest_NeighborDic,cosineSimData,testUsers,weighted_kNN_Dic)







sortKNNS(testUsers)

###By experimenting with different 'K' values, I got the most efficient results in the number 4 and I selected the 'K' value as 4.

decideKNNs(testUsers,4)

findUserVoted(testUsers,userVotedMovies)
meanAE=findMAE(testUsers,userVotedMovies)
print(meanAE)




meanAEWKNN=findMAE_with_Weighted_KNN(testUsers,userVotedMovies,weighted_kNN_Dic)
print(meanAEWKNN)




1.8438724043569243
3.4492864236925262
